# this notebook is to convert/prepare for FIX

The pisa data from fmriprep is the thing to be used - and has to be remolded to fulfull the FSL-type of formatting


Translation recipe:
- **filtered_func_data.nii.gz**
        preprocessed 4D data –> MNI paediatric asymm desc-preproc bold 
            (I personally also applied 6mm smoothing and high band pass filtering)
- **filtered_func_data.ica**  melodic (command-line program) full output directory –> you get this by running single subject ICA
- **mc/prefiltered_func_data_mcf.par** motion parameters created by mcflirt (in mc subdirectory) –> I don’t know what is the par extension, but I should be able to get the motion regressors from the confound regressors tsv file, columns “trans_x”, “trans_y”, “trans_z”, “rot_x”, “rot_y”, "rot_z"
- **mask.nii.gz** valid mask relating to the 4D data –> MNI paediatric asymm desc-brain mask
- **mean_func.nii.gz** temporal mean of 4D data –> you can calculate this with fslmaths T mean I think
- **reg/example_func.nii.gz** example image from 4D data –> given that I don’t need to apply any transformation, should I just use the MNI paediatric asymm desc-preproc bold?
- **reg/highres.nii.gz** brain-extracted structural –> MNI paediatric asymm T1w
- **reg/highres2example_func.mat** FLIRT transform from structural to functional space –> all the images already in MNI paediatric asymmetrical space, so should I replace this with an identity matrix, as suggested here https://github.com/poldracklab/fmriprep/issues/1765 3?
- **reg/FNIRTOUTPUT** I don't know how it's called, but I think FNIRT does output some warpfield that could be applied.
- **design.fsf** FEAT/MELODIC setup file; if present, this controls the default temporal filtering of motion parameters –> I think this is optional


In [10]:
# grab a list of all the BIDS-dirs
# figure out PISA-num and job-num

# make directories, sorted by job, with all of those contents
import glob
import numpy as np
import re
import os
import pandas
import shutil
import datetime
import matplotlib.pyplot as plt
from matplotlib.transforms import Bbox
import nilearn
import imageio
from nilearn import image, plotting


In [11]:
# todo --> I need to grab files from the spm-space, throughout, if that is at all possible
# but for this --> I need to grab the 'boldref' output images; i.e. SMC, STC and SDC already applied
# --> Motion, Slicetiming, Fmap(s)


In [12]:
# %matplotlib qt

In [13]:
paths = glob.glob('jobs/[0-9]*')
# basedir = '/home/johanv/mnt/hpcworking/projects/hmm-pisa/analysis/prepare_fix/'
basedir = '/mnt/lustre/working/lab_michaebr/johanV/projects/hmm-pisa/analysis/prepare_fix/'

In [14]:
brain_mask_func_fname = '{}sbrainmask_func.nii.gz'.format(basedir)    

In [15]:
def link_it(s, t):
    print('linking: {} --> {}'.format(s, t))

    # shutil.copy2(s ,t)
    os.symlink(s, t)
    

    

In [16]:
def make_fig(mov, fd, thr=0.5, shifts=[-1,0,1,2]):
    mov_params_for_fsl = mov
    framewise_displacement = fd

    fig=plt.figure()
    fig.patch.set_facecolor('#FFFFFF')
    plt.subplot(312)
    plt.plot(mov_params_for_fsl[:,0:3] / 3.14 * 180)
    plt.ylabel('degrees')
    plt.xlim(0, 986)
    plt.xticks([])
    plt.title('rotations')
    #plt.ylim(-5, 5)
    plt.subplot(311)
    plt.plot(mov_params_for_fsl[:,3:6])
    plt.ylabel('mm')
    plt.xlim(0, 986)
    plt.xticks([])
    plt.title('translations')
    #plt.ylim(-5, 5)
    plt.subplot(313)
    plt.plot(framewise_displacement)
    plt.ylabel('mm')
    plt.ylim(0, 2)
    plt.xlim(0, 986)
    
    framewise_displacement[0]=0
    
    inds=[]
    for s in shifts:
        inds.extend(np.where(np.roll(framewise_displacement, s) > thr)[0])
    inds=sorted(list(set(inds)))    
    for i in inds:
        plt.vlines(i, 0, 2, color='k')
    plt.hlines(thr, 0, 986, colors='r')
    plt.text(986, 2, 'thr={:.2f}    {} nulled = {:d}%'.format(thr, len(inds), int(100*len(inds)/len(framewise_displacement))), 
            verticalalignment='top',horizontalalignment='right', color='g',
            bbox={'facecolor': 'w', 'alpha': 1.0, 'pad': 3})
    plt.xlabel('scans')
    plt.title('framewise displacement')
    plt.gcf().suptitle('JOB_NUMBER = {}    PISA_NUMBER = {}'.format(job_num, pisa_number))
    
    return plt.gcf()

In [17]:
def make_design_fsf_contents(func_for_melodic, func_mask, t1_for_melodic, nvol, outputdir):
    s_to_return=f'\
\n\
# FEAT version number\n\
set fmri(version) 6.00\n\
\n\
# Are we in MELODIC?\n\
set fmri(inmelodic) 0\n\
\n\
# Analysis level\n\
# 1 : First-level analysis\n\
# 2 : Higher-level analysis\n\
set fmri(level) 1\n\
\n\
# Which stages to run\n\
# 0 : No first-level analysis (registration and/or group stats only)\n\
# 7 : Full first-level analysis\n\
# 1 : Pre-processing\n\
# 2 : Statistics\n\
set fmri(analysis) 1\n\
\n\
# Use relative filenames\n\
set fmri(relative_yn) 0\n\
\n\
# Balloon help\n\
set fmri(help_yn) 1\n\
\n\
# Run Featwatcher\n\
set fmri(featwatcher_yn) 0\n\
\n\
# Cleanup first-level standard-space images\n\
set fmri(sscleanup_yn) 0\n\
\n\
# Output directory\n\
set fmri(outputdir) "{outputdir}"\n\
\n\
# TR(s)\n\
set fmri(tr) 0.820000 \n\
\n\
# Total volumes\n\
set fmri(npts) {nvol}\n\
\n\
# Delete volumes\n\
set fmri(ndelete) 0\n\
\n\
# Perfusion tag/control order\n\
set fmri(tagfirst) 1\n\
\n\
# Number of first-level analyses\n\
set fmri(multiple) 1\n\
\n\
# Higher-level input type\n\
# 1 : Inputs are lower-level FEAT directories\n\
# 2 : Inputs are cope images from FEAT directories\n\
set fmri(inputtype) 2\n\
\n\
# Carry out pre-stats processing?\n\
set fmri(filtering_yn) 1\n\
\n\
# Brain/background threshold, %\n\
set fmri(brain_thresh) 0\n\
\n\
# Critical z for design efficiency calculation\n\
set fmri(critical_z) 5.3\n\
\n\
# Noise level\n\
set fmri(noise) 0.66\n\
\n\
# Noise AR(1)\n\
set fmri(noisear) 0.34\n\
\n\
# Motion correction\n\
# 0 : None\n\
# 1 : MCFLIRT\n\
set fmri(mc) 0\n\
\n\
# Spin-history (currently obsolete)\n\
set fmri(sh_yn) 0\n\
\n\
# B0 fieldmap unwarping?\n\
set fmri(regunwarp_yn) 0\n\
\n\
# EPI dwell time (ms)\n\
set fmri(dwell) 0.7\n\
\n\
# EPI TE (ms)\n\
set fmri(te) 35\n\
\n\
# % Signal loss threshold\n\
set fmri(signallossthresh) 10\n\
\n\
# Unwarp direction\n\
set fmri(unwarp_dir) y-\n\
\n\
# Slice timing correction\n\
# 0 : None\n\
# 1 : Regular up (0, 1, 2, 3, ...)\n\
# 2 : Regular down\n\
# 3 : Use slice order file\n\
# 4 : Use slice timings file\n\
# 5 : Interleaved (0, 2, 4 ... 1, 3, 5 ... )\n\
set fmri(st) 0\n\
\n\
# Slice timings file\n\
set fmri(st_file) ""\n\
\n\
# BET brain extraction\n\
set fmri(bet_yn) 0\n\
\n\
# Spatial smoothing FWHM (mm)\n\
set fmri(smooth) 0\n\
\n\
# Intensity normalization\n\
set fmri(norm_yn) 0\n\
\n\
# Perfusion subtraction\n\
set fmri(perfsub_yn) 0\n\
\n\
# Highpass temporal filtering\n\
set fmri(temphp_yn) 1\n\
\n\
# Lowpass temporal filtering\n\
set fmri(templp_yn) 0\n\
\n\
# MELODIC ICA data exploration\n\
set fmri(melodic_yn) 1\n\
\n\
# Carry out main stats?\n\
set fmri(stats_yn) 0\n\
\n\
# Carry out prewhitening?\n\
set fmri(prewhiten_yn) 1\n\
\n\
# Add motion parameters to model\n\
# 0 : No\n\
# 1 : Yes\n\
set fmri(motionevs) 0\n\
set fmri(motionevsbeta) ""\n\
set fmri(scriptevsbeta) ""\n\
\n\
# Robust outlier detection in FLAME?\n\
set fmri(robust_yn) 0\n\
\n\
# Higher-level modelling\n\
# 3 : Fixed effects\n\
# 0 : Mixed Effects: Simple OLS\n\
# 2 : Mixed Effects: FLAME 1\n\
# 1 : Mixed Effects: FLAME 1+2\n\
set fmri(mixed_yn) 2\n\
\n\
# Number of EVs\n\
set fmri(evs_orig) 1\n\
set fmri(evs_real) 2\n\
set fmri(evs_vox) 0\n\
\n\
# Number of contrasts\n\
set fmri(ncon_orig) 1\n\
set fmri(ncon_real) 1\n\
\n\
# Number of F-tests\n\
set fmri(nftests_orig) 0\n\
set fmri(nftests_real) 0\n\
\n\
# Add constant column to design matrix? (obsolete)\n\
set fmri(constcol) 0\n\
\n\
# Carry out post-stats steps?\n\
set fmri(poststats_yn) 0\n\
\n\
# Pre-threshold masking?\n\
set fmri(threshmask) ""\n\
\n\
# Thresholding\n\
# 0 : None\n\
# 1 : Uncorrected\n\
# 2 : Voxel\n\
# 3 : Cluster\n\
set fmri(thresh) 3\n\
\n\
# P threshold\n\
set fmri(prob_thresh) 0.05\n\
\n\
# Z threshold\n\
set fmri(z_thresh) 2.3\n\
\n\
# Z min/max for colour rendering\n\
# 0 : Use actual Z min/max\n\
# 1 : Use preset Z min/max\n\
set fmri(zdisplay) 0\n\
\n\
# Z min in colour rendering\n\
set fmri(zmin) 2\n\
\n\
# Z max in colour rendering\n\
set fmri(zmax) 8\n\
\n\
# Colour rendering type\n\
# 0 : Solid blobs\n\
# 1 : Transparent blobs\n\
set fmri(rendertype) 1\n\
\n\
# Background image for higher-level stats overlays\n\
# 1 : Mean highres\n\
# 2 : First highres\n\
# 3 : Mean functional\n\
# 4 : First functional\n\
# 5 : Standard space template\n\
set fmri(bgimage) 1\n\
\n\
# Create time series plots\n\
set fmri(tsplot_yn) 1\n\
\n\
# Registration to initial structural\n\
set fmri(reginitial_highres_yn) 0\n\
\n\
# Search space for registration to initial structural\n\
# 0   : No search\n\
# 90  : Normal search\n\
# 180 : Full search\n\
set fmri(reginitial_highres_search) 90\n\
\n\
# Degrees of Freedom for registration to initial structural\n\
set fmri(reginitial_highres_dof) 3\n\
\n\
# Registration to main structural\n\
set fmri(reghighres_yn) 1\n\
\n\
# Search space for registration to main structural\n\
# 0   : No search\n\
# 90  : Normal search\n\
# 180 : Full search\n\
set fmri(reghighres_search) 90\n\
\n\
# Degrees of Freedom for registration to main structural\n\
set fmri(reghighres_dof) BBR\n\
\n\
# Registration to standard image?\n\
set fmri(regstandard_yn) 1\n\
\n\
# Use alternate reference images?\n\
set fmri(alternateReference_yn) 0\n\
\n\
# Standard image\n\
set fmri(regstandard) "/usr/share/fsl/5.0/data/standard/MNI152_T1_2mm_brain"\n\
\n\
# Search space for registration to standard space\n\
# 0   : No search\n\
# 90  : Normal search\n\
# 180 : Full search\n\
set fmri(regstandard_search) 90\n\
\n\
# Degrees of Freedom for registration to standard space\n\
set fmri(regstandard_dof) 12\n\
\n\
# Do nonlinear registration from structural to standard space?\n\
set fmri(regstandard_nonlinear_yn) 1\n\
\n\
# Control nonlinear warp field resolution\n\
set fmri(regstandard_nonlinear_warpres) 6 \n\
\n\
# High pass filter cutoff\n\
set fmri(paradigm_hp) 2000\n\
\n\
# Total voxels\n\
set fmri(totalVoxels) 437547360\n\
\n\
\n\
# Number of lower-level copes feeding into higher-level analysis\n\
set fmri(ncopeinputs) 0\n\
\n\
# 4D AVW data or FEAT directory (1)\n\
set feat_files(1) "{func_for_melodic}"\n\
\n\
# Add confound EVs text file\n\
set fmri(confoundevs) 0\n\
\n\
# Subject\'s structural image for analysis 1\n\
set highres_files(1) "{t1_for_melodic}"\n\
\n\
# EV 1 title\n\
set fmri(evtitle1) ""\n\
\n\
# Basic waveform shape (EV 1)\n\
# 0 : Square\n\
# 1 : Sinusoid\n\
# 2 : Custom (1 entry per volume)\n\
# 3 : Custom (3 column format)\n\
# 4 : Interaction\n\
# 10 : Empty (all zeros)\n\
set fmri(shape1) 0\n\
\n\
# Convolution (EV 1)\n\
# 0 : None\n\
# 1 : Gaussian\n\
# 2 : Gamma\n\
# 3 : Double-Gamma HRF\n\
# 4 : Gamma basis functions\n\
# 5 : Sine basis functions\n\
# 6 : FIR basis functions\n\
set fmri(convolve1) 2\n\
\n\
# Convolve phase (EV 1)\n\
set fmri(convolve_phase1) 0\n\
\n\
# Apply temporal filtering (EV 1)\n\
set fmri(tempfilt_yn1) 1\n\
\n\
# Add temporal derivative (EV 1)\n\
set fmri(deriv_yn1) 1\n\
\n\
# Skip (EV 1)\n\
set fmri(skip1) 0\n\
\n\
# Off (EV 1)\n\
set fmri(off1) 30\n\
\n\
# On (EV 1)\n\
set fmri(on1) 30\n\
\n\
# Phase (EV 1)\n\
set fmri(phase1) 0\n\
\n\
# Stop (EV 1)\n\
set fmri(stop1) -1\n\
\n\
# Gamma sigma (EV 1)\n\
set fmri(gammasigma1) 3\n\
\n\
# Gamma delay (EV 1)\n\
set fmri(gammadelay1) 6\n\
\n\
# Orthogonalise EV 1 wrt EV 0\n\
set fmri(ortho1.0) 0\n\
\n\
# Orthogonalise EV 1 wrt EV 1\n\
set fmri(ortho1.1) 0\n\
\n\
# Contrast & F-tests mode\n\
# real : control real EVs\n\
# orig : control original EVs\n\
set fmri(con_mode_old) orig\n\
set fmri(con_mode) orig\n\
\n\
# Display images for contrast_real 1\n\
set fmri(conpic_real.1) 1\n\
\n\
# Title for contrast_real 1\n\
set fmri(conname_real.1) ""\n\
\n\
# Real contrast_real vector 1 element 1\n\
set fmri(con_real1.1) 1\n\
\n\
# Real contrast_real vector 1 element 2\n\
set fmri(con_real1.2) 0\n\
\n\
# Display images for contrast_orig 1\n\
set fmri(conpic_orig.1) 1\n\
\n\
# Title for contrast_orig 1\n\
set fmri(conname_orig.1) ""\n\
\n\
# Real contrast_orig vector 1 element 1\n\
set fmri(con_orig1.1) 1\n\
\n\
# Contrast masking - use >0 instead of thresholding?\n\
set fmri(conmask_zerothresh_yn) 0\n\
\n\
# Do contrast masking at all?\n\
set fmri(conmask1_1) 0\n\
\n\
##########################################################\n\
# Now options that don\'t appear in the GUI\n\
\n\
# Alternative (to BETting) mask image\n\
set fmri(alternative_mask) "{func_mask}"\n\
\n\
# Initial structural space registration initialisation transform\n\
set fmri(init_initial_highres) ""\n\
\n\
# Structural space registration initialisation transform\n\
set fmri(init_highres) ""\n\
\n\
# Standard space registration initialisation transform\n\
set fmri(init_standard) ""\n\
\n\
# For full FEAT analysis: overwrite existing .feat output dir?\n\
set fmri(overwrite_yn) 0\n'
    return s_to_return

### Make another motion check --> .gif of the raw functionals (!!)
- Allows us to SEE - what the motion is, to gain an impression
- I need to read in/load the entire image data.

In [18]:
# for p in paths[slice(1,2)]:
for p in paths:

    fd = basedir + '../' + p + '/job-01/Data/sub-01/output_dir/fmriprep/sub-01/ses-01/func/'
    
    # func='{}sub-01_ses-01_task-clips_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'.format(fd)
    func='{}sub-01_ses-01_task-clips_desc-preproc_bold.nii.gz'.format(fd)
    confounds='{}sub-01_ses-01_task-clips_desc-confounds_regressors.tsv'.format(fd)
    # funcref='{}sub-01_ses-01_task-clips_space-MNI152NLin2009cAsym_boldref.nii.gz'.format(fd)
    funcref='{}sub-01_ses-01_task-clips_boldref.nii.gz'.format(fd)
    # gm_seg_mask_1000_2000='{}sub-01_ses-01_task-clips_space-MNI152NLin2009cAsym_desc-aparcaseg_dseg.nii.gz'.format(fd)
    # more_brain_seg='{}sub-01_ses-01_task-clips_space-MNI152NLin2009cAsym_desc-aseg_dseg.nii.gz'.format(fd)
    # brain_mask='{}sub-01_ses-01_task-clips_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz'.format(fd)
    func_mask='{}sub-01_ses-01_task-clips_desc-brain_mask.nii.gz'.format(fd)
    t1_mask='{}../../anat/sub-01_desc-brain_mask.nii.gz'.format(fd)
    t1 = '{}../../anat/sub-01_desc-preproc_T1w.nii.gz'.format(fd)
    
    
    # figure out PISA-number
    pisa_number=re.findall('[0-9]{6}', fd)[0]
    # figure out job-number
    job_num = re.findall('_[0-9]{3}',glob.glob(p + '/JOB_*')[0])[0][1:]
    print('\n\nHandling: {}; JobID = {}'.format(pisa_number, job_num))
    
    # make a new directory 
    newdir = 'tofix_hpc/fix_{}/'.format(job_num)
    os.makedirs(newdir, exist_ok=True)
    
    
    link_it(func, newdir + 'func.nii.gz')
    link_it(t1, newdir + 't1.nii.gz')
    link_it(func_mask, newdir + 'func_mask.nii.gz')
    link_it(t1_mask, newdir + 't1_mask.nii.gz')
    
    

    # we need this to run the Melodic preprocessing
    bids_func = '{}../jobs/{}/job-01/Data/sub-01/bids_dir/sub-01/ses-01/func/sub-01_ses-01_task-clips_bold.nii'.format(basedir, pisa_number)
    # t1 = '{}../jobs/{}/job-01/Data/sub-01/bids_dir/sub-01/ses-01/anat/sub-01_ses-01_T1w.nii'.format(basedir, pisa_number)

    
    

    with open(newdir + 'pisa_{}.txt'.format(pisa_number), 'w') as f:
        f.write('')

    
    # 1. handle func data
    # sourcef = func
    # targetf = newdir + 'func_data_link.nii.gz'
    # funclink_fname = basedir + targetf
    # link_it(sourcef, targetf)
    # --> I, then, still need to run the high-pass filtering (fslmaths) + the smoothing (fslmaths) on this.
    # --> Also calculate the mean, then... --> check it, to see what makes sense...
    # Likely, no smoothing is required, since motion corr, dist corr & slice timing corr ahve already been applied.
    
    
    # 2. Handle ica directory
    # filtered_func_data.ica --> run melodic...
    
    
    # handle the motion params...
    df=pandas.read_csv(confounds, sep='\t')
    mov_params_for_fsl = df[['rot_x','rot_y','rot_z','trans_x', 'trans_y', 'trans_z']].values
    framewise_displacement = df['framewise_displacement'].values
    # framewise_displacement = framewise_displacement[18:(18+958),:]
    # os.makedirs(newdir + 'mc', exist_ok=True)
    targetf_mc = newdir + 'prefiltered_func_data_mcf.par'
    np.savetxt(targetf_mc, mov_params_for_fsl[18:(18+958),:])
    print('writing motion params: --> {}'.format(targetf_mc))
    
    
    # do very fanciful analysis on the motion parameters -->
    # how does FD (power) look like? 
    # How many scans will be 'nulled'?
    # how does plot of rotations and translations look like?
    # this could be used to put next to the ICA decomposition(s).
    fig = make_fig(mov_params_for_fsl[18:(18+958),:], framewise_displacement[18:(18+958)])
    title_fig = basedir + newdir + 'mov_job_{}.png'.format(job_num)
    plt.savefig(title_fig, dpi=250)
    plt.close(plt.gcf())
    # linking mean image --> make it!!
    # run fslmaths (within the mask, I suppose)

    
    
    # now; run the gif-maker.
    target_name = newdir + 'motion_picture_job_{}.gif'.format(job_num)
    target_directory = newdir + 'png/'
    
    # in order to make the figure on the cluster...
    # we'd need to make 'make_gif' a function in a text file.
    # then we'd need to make a python script that defines the variables
    # and a PBS script to run the script.
    
    # python script contents:
    framewise_displacement[0]=0
    py_contents = f'\
import sys\n\
sys.path.append(\'{os.getcwd()}\')\n\
\n\
import make_gif\n\
import numpy\n\
import os\n\
from numpy import array\n\
\n\
orig_func = \'{bids_func}\'\n\
\n\
target_name = \'{os.path.join(basedir, target_name)}\'\n\
\n\
framewise_displacement = {framewise_displacement.__repr__()}\n\
\n\
target_directory = \'{os.path.join(basedir, target_directory)}\'\n\
\n\
job_num = \'{job_num}\'\n\
\n\
pisa_number = \'{pisa_number}\'\n\
\n\
make_gif.make_gif(orig_func, target_name, framewise_displacement, target_directory, job_num, pisa_number)\n\
\n\
os.system(\'ffmpeg -f gif -i motion_picture_job_{job_num}.gif motion_picture_job_{job_num}.mp4\')\n\
\n'

    py_filename = f'{newdir}/make_gif_{job_num}.py'
    with open(py_filename, 'w') as f:
        f.write(py_contents)
    
    # make_gif(orig_func, target_name, framewise_displacement, target_directory, job_num, pisa_number, totvols=986)
    
    
    # linking the mask:
    # sourcef = basedir + brain_mask # let's just use the MNI brain mask, not sub-specific ones:
    # sourcef = brain_mask # do the original own's mask here
    # img_mask = newdir + 'mask.nii.gz'
    # link_it(sourcef, img_mask)
    
    
    # linking the example func:
    # os.makedirs(newdir + 'reg', exist_ok=True)
    # sourcef = funcref
    # targetf = newdir + 'reg/example_func.nii.gz'
    # link_it(sourcef, targetf)
    
    # linking the struct::
    # sourcef = orig_t1 # basedir + '2009c-brain.nii.gz'
    # targetf = newdir + 'reg/highres.nii.gz'
    # link_it(sourcef, targetf)

    # the transformation matrix == 1 --> DO NOT NEED!
    # targetf = newdir + 'reg/highres2example_func.mat'
    # np.savetxt(targetf, np.eye(4))
    # print('writing: {}'.format(targetf))
  
    # NVOLS:
    nvol = 958
    
    # prepare what we put into the fslmaths:
    filtered_func_data_fname = '{}filtered_func_data.nii.gz'.format(basedir + newdir) # for later use.
    t1_brain = '{}t1_brain.nii.gz'.format(basedir + newdir) # for later use.
    mean_func_fname = '{}mean_func.nii.gz'.format(basedir + newdir) # also - for later use.
    
    # make pbs script to run --> high-pass filtering
    pbs_contents = f'\
##########################################################################\n\
#\n\
#  Script:    (preprocessing_for_FIX)\n\
#  Author:    Johan van der Meer \n\
#  Created:   {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\n\
#\n\
##########################################################################\n\
#PBS -l mem=20gb,walltime=5:00:00,ncpus=5\n\
\n\
module load fsl\n\
module load miniconda3/current\n\
source activate fmriprep-env\n\
\n\
fslroi {func} {filtered_func_data_fname} 18 {nvol}\n\
\n\
# fslmaths {filtered_func_data_fname} -mas {func_mask} {filtered_func_data_fname}\n\
# \n\
fslmaths {t1} -mas {t1_mask} {t1_brain}\n\
\n\
source deactivate fmriprep-env\n\
module unload miniconda3/current\n\
module unload fsl\n'

    # write is as a .pbs script; for later use:
    os.makedirs(newdir + 'pbs', exist_ok=True)
    pbs_filename = '{}/pbs/pbs_preprocessing_fix_{}.pbs'.format(newdir, job_num)
    
    with open(pbs_filename, 'w+') as f:
        f.write(pbs_contents)


    nvol = nvol
    func_for_melodic = filtered_func_data_fname
    t1_for_melodic = t1_brain
    outputdir = '{}feat_{}'.format(basedir + newdir, job_num)
    if not os.path.exists(outputdir):
        os.mkdir(outputdir)
    
    # make the design.fsf file:
    design_fnf_contents = make_design_fsf_contents(func_for_melodic, func_mask, t1_for_melodic, nvol, outputdir)

    # output='{}melodic_{}.ica'.format(basedir + newdir, job_num)
    # if not os.path.exists(output):
    #     os.mkdir(output)

    
    designfsffilename = os.path.join(basedir, newdir, 'design.fsf')
    with open(designfsffilename,'w') as f:
        f.write(design_fnf_contents)
    
        
        
        
    # designfsf = designfsffilename
    # output = '{}melodic_{}.ica'.format(basedir + newdir, job_num)
    inputdata = func_for_melodic
    
    pbs_contents = f'\
##########################################################################\n\
#\n\
#  Script:    run Melodic (run Melodic ICA)\n\
#  Author:    Johan van der Meer \n\
#  Created:   {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\n\
#\n\
##########################################################################\n\
#PBS -l mem=20gb,walltime=5:00:00,ncpus=5\n\
\n\
module load fsl\n\
module load miniconda3/current\n\
source activate fmriprep-env\n\
\n\
\n\
feat {designfsffilename} -D {outputdir}.feat -I 1 -init\n\
\n\
feat {designfsffilename} -D {outputdir}.feat -I 1 -prestats\n\
\n\
rm {inputdata} -f\n\
\n\
# melodic -i {outputdir}/filtered_func_data.nii.gz -o {outputdir} -v --nobet --bgthreshold=3 --tr=0.820000 --report --guireport=../../report.html -d 0 --mmthresh=0.5  --report_maps=" -S 1 1800 "\n\
\n\
# feat {designfsffilename} -D {outputdir}.feat -stop\n\
\n\
# take care of motion parameters (for FIX feature extraction later on...)\n\
mkdir {outputdir}.feat/mc\n\
cp {targetf_mc} {outputdir}.feat/mc/.\n\
\n\
source deactivate fmriprep-env\n\
module unload miniconda3/current\n\
module unload fsl\n'
    runmelodic_fname = '{}pbs/run_melodic_{}.pbs'.format(basedir + newdir, job_num)
    with open(runmelodic_fname, 'w') as f:
        f.write(pbs_contents)
        
    if not os.path.exists(outputdir+'.feat'):
        os.mkdir(outputdir+'.feat')
    if not os.path.exists(outputdir+'.feat'+'/logs'):
        os.mkdir(outputdir+'.feat'+'/logs')
    
    # os.mkdir(output)


    
    
    
    
    
    



Handling: 471722; JobID = 112
linking: /mnt/lustre/working/lab_michaebr/johanV/projects/hmm-pisa/analysis/prepare_fix/../jobs/471722/job-01/Data/sub-01/output_dir/fmriprep/sub-01/ses-01/func/sub-01_ses-01_task-clips_desc-preproc_bold.nii.gz --> tofix_hpc/fix_112/func.nii.gz
linking: /mnt/lustre/working/lab_michaebr/johanV/projects/hmm-pisa/analysis/prepare_fix/../jobs/471722/job-01/Data/sub-01/output_dir/fmriprep/sub-01/ses-01/func/../../anat/sub-01_desc-preproc_T1w.nii.gz --> tofix_hpc/fix_112/t1.nii.gz
linking: /mnt/lustre/working/lab_michaebr/johanV/projects/hmm-pisa/analysis/prepare_fix/../jobs/471722/job-01/Data/sub-01/output_dir/fmriprep/sub-01/ses-01/func/sub-01_ses-01_task-clips_desc-brain_mask.nii.gz --> tofix_hpc/fix_112/func_mask.nii.gz
linking: /mnt/lustre/working/lab_michaebr/johanV/projects/hmm-pisa/analysis/prepare_fix/../jobs/471722/job-01/Data/sub-01/output_dir/fmriprep/sub-01/ses-01/func/../../anat/sub-01_desc-brain_mask.nii.gz --> tofix_hpc/fix_112/t1_mask.nii.gz
w

In [11]:
pwd

'/home/johanv/mnt/hpcworking/projects/hmm-pisa/analysis/prepare_fix'

In [8]:
# def gif_maker(gif_name,png_dir,gif_indx,num_gifs,dpi=90):
#     # make png path if it doesn't exist already
#     if not os.path.exists(png_dir):
#         os.makedirs(png_dir)

#     # save each .png for GIF
#     # lower dpi gives a smaller, grainier GIF; higher dpi gives larger, clearer GIF
#     plt.savefig(png_dir+'frame_'+str(gif_indx)+'_.png',dpi=dpi)
#     plt.close('all') # comment this out if you're just updating the x,y data

#     if gif_indx==num_gifs-1:
#         # sort the .png files based on index used above
#         images,image_file_names = [],[]
#         for file_name in os.listdir(png_dir):
#             if file_name.endswith('.png'):
#                 image_file_names.append(file_name)       
#         sorted_files = sorted(image_file_names, key=lambda y: int(y.split('_')[1]))

#         # define some GIF parameters

#         frame_length = 0.050 # seconds between frames
#         end_pause = 4 # seconds to stay on last frame
        
#         # loop through files, join them to image array, and write to GIF called 'wind_turbine_dist.gif'
#         for ii in range(0,len(sorted_files)):       
#             file_path = os.path.join(png_dir, sorted_files[ii])
#             if ii==len(sorted_files)-1:
#                 for jj in range(0,int(end_pause/frame_length)):
#                     images.append(imageio.imread(file_path))
#             else:
#                 images.append(imageio.imread(file_path))
#         # the duration is the time spent on each image (1/duration is frame rate)
#         imageio.mimsave(gif_name, images,'GIF',duration=frame_length)


In [9]:
# def make_gif(volume_fname, target_name, fd, target_directory, job_num, pisa_number, totvols=986):

#     # volume_fname = name of the nii - unprocessed - file - to check / make a movie of
#     # target_name = name of the fig image
#     # target_directory = name of the directory in which to put in temporary files

    
#     totvols = image.load_img(volume_fname).shape[3]
#     framewise_displacement = fd # yeah.. we need it; to keep track of where we are.

#     for tvol in range(0, totvols):
        
#         fh=plt.figure(figsize=(15, 7.5))

#         # Framewise Displacement Figure
#         a1=plt.subplot(211)
#         plt.plot(framewise_displacement)
#         plt.ylabel('mm')
#         plt.ylim(0, 2)
#         plt.xlim(0, 986)

#         a1.set_position(Bbox([[0.05,0.75],[0.95,0.95]]))

#         framewise_displacement[0]=0

#         tr=0.82
#         inds=[]
#         shifts=[-1, 0, 1, 2]
#         thr=0.5
#         for s in shifts:
#             inds.extend(np.where(np.roll(framewise_displacement, s) > thr)[0])
#         inds=sorted(list(set(inds)))    
#         for i in inds:
#             plt.vlines(i, 0, 2, color='k')
#         plt.hlines(thr, 0, 986, colors='r')
#         plt.text(986, 2, 'vol = {} ({:02d}m:{:02d}s)   thr={:.2f}    {} nulled = {:d}%'.format(tvol, int(tvol*tr//60), round(tvol*tr%60), thr, len(inds), int(100*len(inds)/len(framewise_displacement))), 
#         verticalalignment='top',horizontalalignment='right', color='g',
#         bbox={'facecolor': 'w', 'alpha': 1.0, 'pad': 3})
#         # plt.xlabel('scans')

#         plt.vlines(tvol, -0.2, 2.2, colors='r', linewidth=3, zorder=100)


#         # then -- the EPI image
#         a2=plt.subplot(212)
#         a2.set_position(Bbox([[0.05,0.05],[0.95,0.70]]))

#         this_image = image.index_img(volume_fname, tvol)

#         plotting.plot_epi(this_image, axes=a2, draw_cross=True,  cut_coords=[8, 0, 0], display_mode='ortho', vmin=50, vmax=1100)

#         fh.suptitle('job number: {}    pisa number: {}'.format(job_num, pisa_number))
        
#         gif_maker(target_name, target_directory ,tvol, totvols, dpi=120)
        
#         plt.close(fh)
        
